In [1]:
from ase.io.vasp import read_vasp
from ase.visualize import view
from numpy import sqrt, ndarray, array
from ase import Atoms
from ase.build import bulk, surface
# For version independency, it could be instead by 'from ase.io.vasp import *'
import spglib
from mymetal.build.film.findprim import my_find_prim
from mymetal.build.film.stretch  import  move_atoms, my_find_num_per_slab
import os
from mymetal.build.film.hydroxyl import passivate_surface_custom

# Using Ovito package to slice the bulk structrue, get the pristine slab model

In [9]:
from ovito.modifiers import SliceModifier
from ovito.io import import_file, export_file
pipeline = import_file('POSCAR')
modifier = SliceModifier(normal=(0, 0, 1), distance = 8, slab_width= 9.6)
pipeline.modifiers.append(modifier)
data = pipeline.compute()
export_file(pipeline, "POSCAR-slab", "vasp")

# Using Mymetal package to get the hydroxylated slab

In [3]:
pristine_slab = read_vasp("./POSCAR-slab")
bulk = read_vasp("./POSCAR")

# 控制盒子z方向的高度
cell = pristine_slab.get_cell()
cell[2, 2] += 5  # 增加z方向的盒子大小
pristine_slab.set_cell(cell, scale_atoms=False)  # 保持原子位置不变

hydroxyl = Atoms('OH', positions=[[0, 0, 0], [0, 0, 0.98]])
view(hydroxyl)
adsorbates = {'Si': hydroxyl, 'O': 'H'}
view(pristine_slab)
hydroxylated_slab = passivate_surface_custom(bulk, pristine_slab, adsorbates, cutoff = 1.9, weights = {'Si': 1.0, 'O': 0.5})
view(hydroxylated_slab)

Totally, dangling bonds of Si: 3
Totally, dangling bonds of O: 23


<Popen: returncode: None args: ['/home/louis/mysoft/env/pyenv/mydft/bin/pyth...>

# Fixed 5 Angstron

In [5]:
from ase.io.vasp import write_vasp
from mymetal.build.film.extrfilm import my_extr_thick
from mymetal.universial.atom.fixatom import fixatoms
write_vasp('./POSCAR-hydroxylated-slab',hydroxylated_slab)

indices = [atom.index for atom in hydroxylated_slab 
           if (atom.position[2] - min(hydroxylated_slab.get_positions()[:,2])) < (my_extr_thick(hydroxylated_slab) - 6.5)]
hydroxylated_slab_fixed = fixatoms(hydroxylated_slab , indices = indices) 
write_vasp('./POSCAR-hydroxylated-slab-fixed', hydroxylated_slab_fixed)